In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.filterwarnings('ignore')

In [2]:
IMAGE_SIZE = (380, 380)
INPUT_SHAPE = (380, 380, 3)

In [3]:
test_df = pd.read_csv('data/test.csv')
test_df['category'] = test_df['category'].astype(str)

In [5]:
model = load_model('output/models/efficientnetb4.h5')

In [6]:
image_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)

test_generator=image_datagen.flow_from_dataframe(
    dataframe=test_df, 
    x_col='filename',
    y_col='category',
    directory='data/test',
    class_mode='categorical', 
    color_mode='rgb',
    target_size=IMAGE_SIZE, 
    batch_size=1,
    shuffle=False,
    seed=42
)

Found 12186 validated image filenames belonging to 1 classes.


In [7]:
# reset the generator to start from the first batch to ensure the predictions correspond with the file names
test_generator.reset()

# predict classes
predictions = model.predict_generator(test_generator, steps=test_generator.n, verbose=1)

12186/12186 [==============================] - 7061s 579ms/step


In [8]:
# format result
predicted_class_indices = np.argmax(predictions, axis=1)
indices_classes_df = pd.read_csv('output/indices_classes_mapping.csv')
indices_to_classes = dict(zip(indices_classes_df.iloc[:,0], indices_classes_df.iloc[:,1]))
predicted_classes = np.vectorize(indices_to_classes.get)(predicted_class_indices)
filenames_to_classes = list(zip(test_generator.filenames, predicted_classes))

# save result
result = pd.DataFrame(filenames_to_classes, columns=['filename', 'category'])
result.to_csv('output/result.csv', index=False)